# Using the Arduino Nano for Chamber Control

## Pinout for the Nano

2022-05-11

![Nano](Arduino-Nano-Pinout.jpg)

## Initial Wiring

While connecting up the Nano using breadboard, the button and corresponding LED (will soon control the fan) came on during the inital software development. After some debugging, it seems to be an issue with usign digital pin 4. When I switch to digital pin 5 for detecting the button press, all is well. Below is the initial breadboard wiring diagram:

<img src="chanber_controller_wiring_m01.jpg" alt="Wiring Diagram M01" width="1000"/>

## Initial Code

```
#include <Arduino.h>
#include <Adafruit_SSD1306.h>
#include <Adafruit_I2CDevice.h>
#include <DHT.h>
#include <Adafruit_Sensor.h>
#include <Wire.h>


#define buttonLights 2  //input of the light button
#define buttonVent 3   //input of the vents button
#define buttonFan 5     //input for the fan button
#define LEDlights A0
#define LEDVent A1
#define LEDFan A2
#define temp 0
#define hum 1
#define sensorInterval 10000 //read every 10
#define debounceDelay 50 //pushbutton interval

#define DHTPIN 6  //digital pin for temp and humidity readings
#define DHTTYPE DHT22  //The type of dht probe we are using

#define SCREEN_WIDTH 128 // OLED display width, in pixels
#define SCREEN_HEIGHT 32 // OLED display height, in pixels

// Declaration for an SSD1306 display connected to I2C (SDA, SCL pins)
#define OLED_RESET     4 // Reset pin # (or -1 if sharing Arduino reset pin)

Adafruit_SSD1306 display(SCREEN_WIDTH, SCREEN_HEIGHT, &Wire, OLED_RESET);

DHT dht(DHTPIN, DHTTYPE);

byte readingButtonLights;
byte stateButtonLights;
byte lastStateButtonLights = HIGH;
byte stateLights = LOW;

byte readingButtonFan;
byte stateButtonFan;
byte lastStateButtonFan = HIGH;
byte stateFan = LOW;

byte readingButtonVent;
byte stateButtonVent;
byte lastStateButtonVent = HIGH;
byte stateVent = LOW;

unsigned long sensorCycleStart;
unsigned long sensorCycleCurrent;
unsigned long lastDebounceTimeLights;
unsigned long lastDebounceTimeFan;
unsigned long lastDebounceTimeVent;



void setup() {
  // Start Serial
  Serial.begin(9600);

  //Setup pins
  pinMode(buttonLights, INPUT);
  pinMode(buttonVent, INPUT);
  pinMode(buttonFan, INPUT);

  pinMode(LEDlights, OUTPUT);
  pinMode(LEDFan, OUTPUT);
  pinMode(LEDVent, OUTPUT);

  pinMode(13, OUTPUT);   //board status
  
  //setup loop details
  sensorCycleStart = millis();

  //Setup display
  // SSD1306_SWITCHCAPVCC = generate display voltage from 3.3V internally
  if(!display.begin(SSD1306_SWITCHCAPVCC, 0x3C)) { // Address 0x3C for 128x32
    Serial.println(F("SSD1306 allocation failed"));
    for(;;); // Don't proceed, loop forever
  }
  
  display.display();
  display.clearDisplay();
  delay(200);
  display.setTextSize(1);      // Normal 1:1 pixel scale
  display.setTextColor(SSD1306_WHITE); // Draw white text
  display.setCursor(0, 0);     // Start at top-left corner
  display.write("Hello World");
  display.display();

  //setup dht
  dht.begin();
 
}

void loop() {
  // put your main code here, to run repeatedly:

 //Determine if we need a sensor read
  if (millis()-sensorCycleStart > sensorInterval){
    //Read temp and humidity
    float t = dht.readTemperature();
    float h = dht.readHumidity();
    
    //Display temp and humidity
    display.clearDisplay();
    display.setTextSize(1);
    display.setCursor(0,0);
    
    char temperature[5];
    dtostrf(t,5,1,temperature);
    
    display.write("Temp: ");
    display.write(temperature);
    display.display();

    char humidity[5];
    dtostrf(h,5,1,humidity);

    display.setCursor(0,10);
    display.write("Humid: ");
    display.write(humidity);
    display.display();
    
    
    

    //update the display

    //reset sensorCycleStart
    sensorCycleStart = millis();
    Serial.println("Sensor Read Triggered");
    if (digitalRead(13)==HIGH){
      digitalWrite(13, LOW);
      //digitalWrite(LEDlights, LOW);
    }else{
      digitalWrite(13, HIGH);
      //digitalWrite(LEDlights, HIGH);
    }
  }

  //Evaluate button pushes

  {//Lights

  readingButtonLights = digitalRead(buttonLights);
  // check to see if you just pressed the button
  // (i.e. the input went from LOW to HIGH), and you've waited long enough
  // since the last press to ignore any noise:

  // If the switch changed, due to noise or pressing:
  if (readingButtonLights != lastStateButtonLights) {
    // reset the debouncing timer
    lastDebounceTimeLights = millis();
  }

  if ((millis() - lastDebounceTimeLights) > debounceDelay) {
    // whatever the reading is at, it's been there for longer than the debounce
    // delay, so take it as the actual current state:

    // if the button state has changed:
    if (readingButtonLights != stateButtonLights) {
      stateButtonLights = readingButtonLights;

      //toggle the LED
      if (stateButtonLights == HIGH){
        stateLights = !stateLights;
      }

      display.clearDisplay();
      display.setCursor(0,0);
      if (stateLights == HIGH){
        display.write("Lights On");
      }else{
        display.write("Lights Off");
      }
      display.display();
    }
  }
  
  //Set the lights state
  digitalWrite(LEDlights, stateLights);

  //save the reading. Next time through the loop, it'll be the lastButtonState:
  lastStateButtonLights = readingButtonLights;
  }
  
  {//Fan

  readingButtonFan = digitalRead(buttonFan);
  Serial.print(readingButtonFan);
  // check to see if you just pressed the button
  // (i.e. the input went from LOW to HIGH), and you've waited long enough
  // since the last press to ignore any noise:

  // If the switch changed, due to noise or pressing:
  if (readingButtonFan != lastStateButtonFan) {
    // reset the debouncing timer
    lastDebounceTimeFan = millis();
  }

  if ((millis() - lastDebounceTimeFan) > debounceDelay) {
    // whatever the reading is at, it's been there for longer than the debounce
    // delay, so take it as the actual current state:

    // if the button state has changed:
    if (readingButtonFan != stateButtonFan) {
      stateButtonFan = readingButtonFan;

      //toggle the LED
      if (stateButtonFan == HIGH){
        stateFan = !stateFan;
      }

      display.clearDisplay();
      display.setCursor(0,0);
      if (stateFan == HIGH){
        display.write("Fan On");
      }else{
        display.write("Fan Off");
      }
      display.display();
    }
  }

  //Set the lights state
  digitalWrite(LEDFan, stateFan);

  //save the reading. Next time through the loop, it'll be the lastButtonState:
  lastStateButtonFan = readingButtonFan;
  }

  {// Vent Button
  readingButtonVent = digitalRead(buttonVent);
  // check to see if you just pressed the button
  // (i.e. the input went from LOW to HIGH), and you've waited long enough
  // since the last press to ignore any noise:

  // If the switch changed, due to noise or pressing:
  if (readingButtonVent != lastStateButtonVent) {
    // reset the debouncing timer
    lastDebounceTimeVent = millis();
  }

  if ((millis() - lastDebounceTimeVent) > debounceDelay) {
    // whatever the reading is at, it's been there for longer than the debounce
    // delay, so take it as the actual current state:

    // if the button state has changed:
    if (readingButtonVent != stateButtonVent) {
      stateButtonVent = readingButtonVent;

      //toggle the LED
      if (stateButtonVent == HIGH){
        stateVent = !stateVent;
      }

      display.clearDisplay();
      display.setCursor(0,0);
      if (stateVent == HIGH){
        display.write("Vent On");
      }else{
        display.write("Vent Off");
      }
      display.display();
    }
  }

  //Set the lights state
  digitalWrite(LEDVent, stateVent);
    
  // save the reading. Next time through the loop, it'll be the lastButtonState:
  lastStateButtonVent = readingButtonVent;
  }
  //delay(2000);
}
```

## Graph Development
2022-05-11

```
void testdrawline() {
  int16_t i;

  display.clearDisplay(); // Clear display buffer

  //draw x axis
  display.drawLine(0, 31, 127, 31, SSD1306_WHITE);

  //draw y axis
  display.drawLine(0, 0, 0, 31, SSD1306_WHITE);

  //do the drawing
  display.display();

  for (int i=0; i<128; i++) {
    
    y = random(30,40);
    yplot = 31.0-(y-20.0)/(80.0-20.0)*31.0*i/50.0;
    display.fillCircle(i, yplot, 1, SSD1306_WHITE);
    display.display();
  }
}
```

## Graphing Algorithm
2022-05-12

* Set initial time
* check if ncount is 0, if so, read temp and humidty
* If 60 sec has elapsed, read temp and humidity
* If n element array count less than n
    * Add Temp and humidity to corresponding n element arrays
    * else shift all elements down on spot so 0th element becomes 1st element, the n-1 element becomes the nth element.
    * then add temp and humidity to corresponding n element arrays
    * if ncount = n, stop incrementing
* based on the count, plot the array elements, x is based on ncount*y where y is some integer used to spread out the points on the x axis
* display the T and H as T:XX.X and H:YY.Y
* Reset SensorCycleCount

## Programming update on Arduino Nano Pin 4 issue
2022-05-12

* All buttons and graphing routines are working
* After reviewing the code, **Pin 4** issue is really due to the fact that the OLED screen is using that for **Reset**. We could pick another pin or simply let it be.

## Thoughts on how to control vents for chamber
2022-05-13

* The plans is to use an attiny84 as the microprocessor for vent control
* The attiny84 will be tied to main microprocessor which will be some form of ATMEGA328P
    * Some form because it could be an Arduino Nano or
    * a bare ATMEGA329P chip.
* The key to doing this is using the TinyWireS library for the Arduino Framework. This library is locate [here](https://github.com/nadavmatalon/TinyWireS).
* The TinyWireS library allows hooking both vents to the 328P for control under I2C.
* The command can be something simply like sending "0" for closed or "1" for open.

### Attiny84 pin out
<img src="ATtiny_x4.png" alt="Attiny84 Pin Diagram" width="1000"/>

### Example code for the Slave

```
/*
    TinyWireS Library: ATtiny84A-Side (Slave)
    -----------------------------------------
    INTRODUCTION
    ------------
    The following sketch provides a simple example of setting up an ATtiny84A as an I2C Slave device
    on the I2C bus and an Arduino Uno as the I2C Master.
    At runtime, the Arduino Master will request 4 bytes from the ATtiny84A Slave and then print them
    out using the Serial Monitor.
    HW SETUP
    --------
    - Connect Arduino Pin A4 (HW SDA) to ATtiny84A Pin PA6 (SW SDA) with a 2K2 pull-up resistor to 5V
    - Connect Arduino Pin A5 (HW SCL) to ATtiny84A Pin PA4 (SW SCL) with a 2K2 pull-up resistor to 5V
    - Follow the basic hookup for Arduino & ATtiny84A (Common ground for both ICs, 100nF decoupling capacitors,
      10K pullup resistors for RESET pins, and if needed, external crystals)
    RUNNING THE SKETCH
    ------------------
    Upload this sketch to the ATtiny84A and the other example sketch (Arduino_Uno_Master.ino) to the
    Arduino. Then open the Serial Monitor (make sure the Baud Rate is set to 9600).
    BUG REPORTS
    -----------
    Please report any bugs/issues/suggestions at the GITHUB Repository of this library at:
    https://github.com/nadavmatalon/TinyWireS
    LICENSE
    -------
    The MIT License (MIT)
    Copyright (c) 2016 Nadav Matalon
    Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated
    documentation files (the "Software"), to deal in the Software without restriction, including without
    limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of
    the Software, and to permit persons to whom the Software is furnished to do so, subject to the following
    conditions:
    The above copyright notice and this permission notice shall be included in all copies or substantial
    portions of the Software.
    THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT
    LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT.
    IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY,
    WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE
    SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
*/

#define __AVR_ATtiny84__

#include <Arduino.h>

#if defined(__AVR_ATtiny841__)
#define F_CPU 16000000                          // clock speed: 16MHz (external crystal)
#include "WireS.h"                              // I2C library for ATtiny841 (and other modern ATtinys)
#else
#define F_CPU 20000000                          // clock speed: 20MHz (external crystal)
#include "TinyWireS.h"                          // I2C library for ATtiny84A (and other older ATtinys)
#endif

const byte SLAVE_ADDR = 100;

const byte NUM_BYTES = 4;

volatile byte data[NUM_BYTES] = { 0, 1, 2, 3 };

void setup() {
    TinyWireS.begin(SLAVE_ADDR);
    TinyWireS.onRequest(requestISR);
}

void loop() {}

void requestISR() {
    for (byte i=0; i<NUM_BYTES; i++) {
        TinyWireS.write(data[i]);
        data[i] += 1;
    }
}
```

### Example Code Master

```
/*
    TinyWireS Library: Arduino-Uno-Side (Master)
    --------------------------------------------

    INTRODUCTION
    ------------
    The following sketch provides a simple example of setting up an ATtiny84A as an I2C Slave device 
    on the I2C bus and an Arduino Uno as the I2C Master.

    At runtime, the Arduino Master will request 4 bytes from the ATtiny84A Slave and then print them 
    out using the Serial Monitor.

    HW SETUP
    --------
    - Connect Arduino Pin A4 (HW SDA) to ATtiny84A Pin PA6 (SW SDA) with a 2K2 pull-up resistor to 5V
    - Connect Arduino Pin A5 (HW SCL) to ATtiny84A Pin PA4 (SW SCL) with a 2K2 pull-up resistor to 5V
    - Follow the basic hookup for Arduino & ATtiny84A (Common Ground for both ICs, 100nF decoupling capacitors,
      10K pullup resistors for RESET pins, and if needed, external crystals)

    RUNNING THE SKETCH
    ------------------
    Upload this sketch to the Arduino Uno and the other example sketch (ATtiny84A_Slave.ino) to the
    ATtiny84A. Then open the Serial Monitor (make sure the Baud Rate is set to 9600).

    BUG REPORTS
    -----------
    Please report any bugs/issues/suggestions at the GITHUB Repository of this library at:
    https://github.com/nadavmatalon/TinyWireS

    LICENSE
    -------
    The MIT License (MIT)
    Copyright (c) 2016 Nadav Matalon

    Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated
    documentation files (the "Software"), to deal in the Software without restriction, including without
    limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of
    the Software, and to permit persons to whom the Software is furnished to do so, subject to the following
    conditions:

    The above copyright notice and this permission notice shall be included in all copies or substantial
    portions of the Software.

    THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT
    LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT.
    IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY,
    WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE
    SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
*/

#include <Arduino.h>
#include <Wire.h>

const byte SLAVE_ADDR = 100;

const byte NUM_BYTES = 4;

byte data[NUM_BYTES] = { 0 };

byte bytesReceived = 0;

unsigned long timeNow = millis();

void setup() {
    Serial.begin(9600);
    Wire.begin();
    Serial.print(F("\n\nSerial is Open\n\n"));
}

void loop() {
    if (millis() - timeNow >= 750) {                                        // trigger every 750mS
        Wire.requestFrom(SLAVE_ADDR, NUM_BYTES);                            // request bytes from slave
        bytesReceived = Wire.available();                                   // count how many bytes received
        if (bytesReceived == NUM_BYTES) {                                   // if received correct number of bytes...
            for (byte i=0; i<NUM_BYTES; i++) data[i] = Wire.read();         // read and store each byte
            printData();                                                    // print the received data
        } else {                                                            // if received wrong number of bytes...
            Serial.print(F("\nRequested "));                                // print message with how many bytes received
            Serial.print(NUM_BYTES);
            Serial.print(F(" bytes, but got "));
            Serial.print(bytesReceived);
            Serial.print(F(" bytes\n"));
        }
        timeNow = millis();                                                 // mark preset time for next trigger
    }
}

void printData() {
    Serial.print(F("\n"));
    for (byte i=0; i<NUM_BYTES; i++) {
          Serial.print(F("Byte["));
          Serial.print(i);
          Serial.print(F("]: "));
          Serial.print(data[i]);
          Serial.print(F("\t"));
    }
    Serial.print(F("\n"));    
}
```

### This code is currently sorted on the Asus Computer running VirtualBox.

### How the Attiny84 will be programmed. We will use Spence Kondi library and these are the Arduino settings below:

<img src="PXL_20220513_173216876.jpg" alt="Attiny84 Pin Diagram" width="1000"/>